In [1]:
import pandas as pd
import numpy as np
from datetime import date
import numpy_financial as npf

In [2]:
def amortization_table(interest_rate, years, payments_year, principal, addl_principal=0, start_date=date.today()):
    """ Calculate the amortization schedule given the loan details
    
     Args:
        interest_rate: The annual interest rate for this loan
        years: Number of years for the loan
        payments_year: Number of payments in a year
        principal: Amount borrowed
        addl_principal (optional): Additional payments to be made each period. Assume 0 if nothing provided.
                                   must be a value less then 0, the function will convert a positive value to
                                   negative
        start_date (optional): Start date. Will start on first of next month if none provided

    Returns:
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    # Ensure the additional payments are negative
    if addl_principal > 0:
        addl_principal = -addl_principal
    
    # Create an index of the payment dates
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment_Date"
    
    # Build up the Amortization schedule as a DataFrame
    df = pd.DataFrame(index=rng,columns=['Payment', 'Principal', 'Interest', 
                                         'Addl_Principal', 'Curr_Balance'], dtype='float')
    
    # Add index by period (start at 1 not 0)
    df.reset_index(inplace=True)
    df.index += 1
    df.index.name = "Period"
    
    # Calculate the payment, principal and interests amounts using built in Numpy functions
    per_payment = npf.pmt(interest_rate/payments_year, years*payments_year, principal)
    df["Payment"] = per_payment
    df["Principal"] = npf.ppmt(interest_rate/payments_year, df.index, years*payments_year, principal)
    df["Interest"] = npf.ipmt(interest_rate/payments_year, df.index, years*payments_year, principal)
        
    # Round the values
    df = df.round(2) 
    
    # Add in the additional principal payments
    df["Addl_Principal"] = addl_principal
    
    # Store the Cumulative Principal Payments and ensure it never gets larger than the original principal
    df["Cumulative_Principal"] = (df["Principal"] + df["Addl_Principal"]).cumsum()
    df["Cumulative_Principal"] = df["Cumulative_Principal"].clip(lower=-principal)
    
    # Calculate the current balance for each period
    df["Curr_Balance"] = principal + df["Cumulative_Principal"]
    
    # Determine the last payment date
    try:
        last_payment = df.query("Curr_Balance <= 0")["Curr_Balance"].idxmax(axis=1, skipna=True)
    except ValueError:
        last_payment = df.last_valid_index()
    
    last_payment_date = "{:%m-%d-%Y}".format(df.loc[last_payment, "Payment_Date"])
        
    # Truncate the data frame if we have additional principal payments:
    if addl_principal != 0:
                
        # Remove the extra payment periods
        df = df.loc[0:last_payment].copy()
        
        # Calculate the principal for the last row
        df.loc[last_payment, "Principal"] = -(df.loc[last_payment-1, "Curr_Balance"])
        
        # Calculate the total payment for the last row
        df.loc[last_payment, "Payment"] = df.loc[last_payment, ["Principal", "Interest"]].sum()
        
        # Zero out the additional principal
        df.loc[last_payment, "Addl_Principal"] = 0
        
    # Get the payment info into a DataFrame in column order
    payment_info = (df[["Payment", "Principal", "Addl_Principal", "Interest"]]
                    .sum().to_frame().T)
       
    # Format the Date DataFrame
    payment_details = pd.DataFrame.from_dict(dict([('payoff_date', [last_payment_date]),
                                               ('Interest Rate', [interest_rate]),
                                               ('Number of years', [years])
                                              ]))
    # Add a column showing how much we pay each period.
    # Combine addl principal with principal for total payment
    payment_details["Period_Payment"] = round(per_payment, 2) + addl_principal
    
    payment_summary = pd.concat([payment_details, payment_info], axis=1)
    return df, payment_summary
    

# 30 year amortization schedule

In [3]:
interest_rate = 0.04
years = 30
payments_year = 12 
principal = 1000000

In [4]:
schedule1, stats1 = amortization_table(interest_rate, years, payments_year, principal)

## interest amount paid every month

In [5]:
print(list(schedule1.Interest)) # Payment_Date 	Payment 	Principal 	Interest 	Addl_Principal 	Curr_Balance 	Cumulative_Principal

[-3333.33, -3328.53, -3323.71, -3318.88, -3314.03, -3309.16, -3304.28, -3299.38, -3294.46, -3289.53, -3284.58, -3279.61, -3274.63, -3269.63, -3264.62, -3259.59, -3254.54, -3249.47, -3244.39, -3239.29, -3234.18, -3229.04, -3223.89, -3218.72, -3213.54, -3208.34, -3203.12, -3197.88, -3192.63, -3187.36, -3182.07, -3176.76, -3171.43, -3166.09, -3160.73, -3155.35, -3149.96, -3144.54, -3139.11, -3133.66, -3128.19, -3122.71, -3117.2, -3111.68, -3106.14, -3100.58, -3095.0, -3089.4, -3083.79, -3078.15, -3072.5, -3066.83, -3061.13, -3055.42, -3049.69, -3043.95, -3038.18, -3032.39, -3026.59, -3020.76, -3014.92, -3009.05, -3003.17, -2997.27, -2991.34, -2985.4, -2979.44, -2973.46, -2967.45, -2961.43, -2955.39, -2949.33, -2943.24, -2937.14, -2931.02, -2924.87, -2918.71, -2912.52, -2906.32, -2900.09, -2893.85, -2887.58, -2881.29, -2874.98, -2868.65, -2862.3, -2855.92, -2849.53, -2843.11, -2836.68, -2830.22, -2823.74, -2817.24, -2810.72, -2804.17, -2797.6, -2791.02, -2784.41, -2777.77, -2771.12, -2764.

## monthly payment

In [6]:
print(list(schedule1.Payment))

[-4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15, -4774.15,

## monthly principal paid

In [7]:
print(list(schedule1.Principal))

[-1440.82, -1445.62, -1450.44, -1455.28, -1460.13, -1464.99, -1469.88, -1474.78, -1479.69, -1484.63, -1489.57, -1494.54, -1499.52, -1504.52, -1509.53, -1514.57, -1519.61, -1524.68, -1529.76, -1534.86, -1539.98, -1545.11, -1550.26, -1555.43, -1560.61, -1565.82, -1571.04, -1576.27, -1581.53, -1586.8, -1592.09, -1597.39, -1602.72, -1608.06, -1613.42, -1618.8, -1624.2, -1629.61, -1635.04, -1640.49, -1645.96, -1651.45, -1656.95, -1662.47, -1668.02, -1673.58, -1679.15, -1684.75, -1690.37, -1696.0, -1701.66, -1707.33, -1713.02, -1718.73, -1724.46, -1730.21, -1735.97, -1741.76, -1747.57, -1753.39, -1759.24, -1765.1, -1770.98, -1776.89, -1782.81, -1788.75, -1794.72, -1800.7, -1806.7, -1812.72, -1818.76, -1824.83, -1830.91, -1837.01, -1843.14, -1849.28, -1855.44, -1861.63, -1867.83, -1874.06, -1880.31, -1886.58, -1892.86, -1899.17, -1905.5, -1911.86, -1918.23, -1924.62, -1931.04, -1937.47, -1943.93, -1950.41, -1956.91, -1963.44, -1969.98, -1976.55, -1983.14, -1989.75, -1996.38, -2003.03, -2009.7

In [8]:
print(list(schedule1.Curr_Balance))

[998559.18, 997113.56, 995663.12, 994207.84, 992747.71, 991282.72, 989812.84, 988338.06, 986858.37, 985373.74, 983884.17, 982389.63, 980890.11, 979385.59, 977876.06, 976361.49, 974841.88, 973317.2, 971787.44, 970252.58, 968712.6, 967167.49, 965617.23, 964061.8, 962501.19, 960935.37, 959364.33, 957788.06, 956206.53, 954619.73, 953027.64, 951430.25, 949827.53, 948219.47, 946606.05, 944987.25, 943363.05, 941733.44, 940098.4, 938457.91, 936811.95, 935160.5, 933503.55, 931841.08, 930173.06, 928499.48, 926820.33, 925135.58, 923445.21, 921749.21, 920047.55, 918340.22, 916627.2, 914908.47, 913184.01, 911453.8, 909717.83, 907976.07, 906228.5, 904475.11, 902715.87, 900950.77, 899179.79, 897402.9, 895620.09, 893831.34, 892036.62, 890235.9199999999, 888429.22, 886616.5, 884797.74, 882972.91, 881142.0, 879304.99, 877461.85, 875612.57, 873757.13, 871895.5, 870027.6699999999, 868153.61, 866273.3, 864386.72, 862493.86, 860594.69, 858689.19, 856777.3300000001, 854859.1, 852934.48, 851003.44, 849065.97,

# Total interest paid

In [10]:
print(sum(list(schedule1.Interest)))

-718695.0999999999
